In [1]:
import numpy as np
import tensorflow as tf


from src.loaders.data_loader import load_train_data, load_submission_data
from src.preprocessors.feature_engineering import feature_engineering_cnn
from src.preprocessors.dataset_creators import create_cnn_dataset
from src.preprocessors.splitors import split_train_valid_test_for_cv
from src.model.multiple_output.convolution import Convolution2DVarious
from src.loaders.experiment_loader import load_experiment_cnn
from src.trainers import compile_and_fit_with_pinball_loss
from src.make_submissions.create_submission import create_submission_using_cnn_model
from src.make_submissions.create_submission import evaluate_with_submission_cnn

setting_index = 4
experiment_setting = load_experiment_cnn(setting_index)

days = experiment_setting["days"]
selector = experiment_setting["columns"]

np.random.seed(42)
tf.random.set_seed(42)

In [2]:
df = load_train_data()
df = feature_engineering_cnn(df)

history = {
    "total_error": 0,
    "submission_df": [],
    "total_submission_error": 0,
}

for index, (train, valid, test) in enumerate(split_train_valid_test_for_cv(df, 0.8)):
    if index == 0: continue
    history[index] = {}
    
    window_generator = create_cnn_dataset(train, valid, test, days, selector)
    submission_df = load_submission_data()
    
    for i in range(1, 10):
        q = i/10
        history[index][q] = {"cnn":[], "error": []}
        
        conv_various = Convolution2DVarious(days, 96, len(selector))
        compile_and_fit_with_pinball_loss(conv_various, window_generator, q)
        
        history["total_error"] += conv_various.evaluate(window_generator.test, verbose=0)[0]
        
        submission_df[f"q_{q}"] = create_submission_using_cnn_model(selector, conv_various, train, days)
        history["total_submission_error"] += evaluate_with_submission_cnn(q, train, conv_various, selector, days)
        
    history["submission_df"].append(submission_df)
    submission_df.to_csv(f'cnn_split_dataset_cnn_setting_index_{setting_index}_{index}.csv', index=False)
    
import json
experiment_setting["total_error"] = history["total_error"] / (18 * df.shape[0] * 0.2)
experiment_setting["total_submission_error"] = history["total_submission_error"] / (18 * 80 * 96)

with open(f"experiments/cnn/{setting_index}.json", "w") as f:
    json.dump(experiment_setting, f, indent=4)

Epoch 1/1000
1309/1309 [==============================] - 32s 17ms/step - loss: 1.5935 - mean_absolute_error: 14.8536 - mean_squared_error: 703.9500 - _pinball_loss: 1.5935 - val_loss: 1.5508 - val_mean_absolute_error: 13.0413 - val_mean_squared_error: 529.5299 - val__pinball_loss: 1.5507
Epoch 2/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 1.4043 - mean_absolute_error: 11.8593 - mean_squared_error: 449.2941 - _pinball_loss: 1.4043 - val_loss: 1.5408 - val_mean_absolute_error: 12.8404 - val_mean_squared_error: 522.2010 - val__pinball_loss: 1.5410
Epoch 3/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 1.3889 - mean_absolute_error: 11.6692 - mean_squared_error: 441.1459 - _pinball_loss: 1.3889 - val_loss: 1.5332 - val_mean_absolute_error: 12.8402 - val_mean_squared_error: 525.1038 - val__pinball_loss: 1.5333
Epoch 4/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 1.3804 - mean_absolute_error: 11.5726 - mean_squared_

1309/1309 [==============================] - 10s 8ms/step - loss: 2.1664 - mean_absolute_error: 7.8485 - mean_squared_error: 216.0573 - _pinball_loss: 2.1664 - val_loss: 2.6242 - val_mean_absolute_error: 8.9175 - val_mean_squared_error: 266.8804 - val__pinball_loss: 2.6233
Epoch 7/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 2.1578 - mean_absolute_error: 7.7944 - mean_squared_error: 214.2730 - _pinball_loss: 2.1578 - val_loss: 2.6201 - val_mean_absolute_error: 8.9507 - val_mean_squared_error: 269.0602 - val__pinball_loss: 2.6199
Epoch 8/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 2.1526 - mean_absolute_error: 7.7831 - mean_squared_error: 214.2581 - _pinball_loss: 2.1526 - val_loss: 2.6185 - val_mean_absolute_error: 8.9054 - val_mean_squared_error: 266.7247 - val__pinball_loss: 2.6181
Epoch 9/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 2.1462 - mean_absolute_error: 7.7875 - mean_squared_error: 215.2318 - _pi

1309/1309 [==============================] - 10s 8ms/step - loss: 2.5898 - mean_absolute_error: 6.2066 - mean_squared_error: 143.1402 - _pinball_loss: 2.5898 - val_loss: 3.2712 - val_mean_absolute_error: 7.5548 - val_mean_squared_error: 192.9008 - val__pinball_loss: 3.2719
Epoch 7/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 2.5792 - mean_absolute_error: 6.1886 - mean_squared_error: 142.8990 - _pinball_loss: 2.5792 - val_loss: 3.2695 - val_mean_absolute_error: 7.4769 - val_mean_squared_error: 189.5372 - val__pinball_loss: 3.2690
Epoch 8/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 2.5645 - mean_absolute_error: 6.1385 - mean_squared_error: 141.3865 - _pinball_loss: 2.5645 - val_loss: 3.2572 - val_mean_absolute_error: 7.4811 - val_mean_squared_error: 189.5649 - val__pinball_loss: 3.2575
Epoch 9/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 2.5549 - mean_absolute_error: 6.1008 - mean_squared_error: 140.2620 - _pi

1309/1309 [==============================] - 11s 8ms/step - loss: 2.7037 - mean_absolute_error: 5.4398 - mean_squared_error: 126.0244 - _pinball_loss: 2.7037 - val_loss: 3.5365 - val_mean_absolute_error: 6.9470 - val_mean_squared_error: 176.9909 - val__pinball_loss: 3.5372
Epoch 8/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 2.6873 - mean_absolute_error: 5.3984 - mean_squared_error: 125.1159 - _pinball_loss: 2.6873 - val_loss: 3.5414 - val_mean_absolute_error: 6.9759 - val_mean_squared_error: 176.6033 - val__pinball_loss: 3.5413
Epoch 9/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 2.6830 - mean_absolute_error: 5.3924 - mean_squared_error: 124.7482 - _pinball_loss: 2.6830 - val_loss: 3.5204 - val_mean_absolute_error: 6.9853 - val_mean_squared_error: 174.0689 - val__pinball_loss: 3.5199
Epoch 10/1000
1309/1309 [==============================] - 10s 8ms/step - loss: 2.6742 - mean_absolute_error: 5.3745 - mean_squared_error: 124.1065 - _p

1309/1309 [==============================] - 11s 8ms/step - loss: 2.8268 - mean_absolute_error: 5.6536 - mean_squared_error: 140.4179 - _pinball_loss: 2.8268 - val_loss: 3.6469 - val_mean_absolute_error: 7.2937 - val_mean_squared_error: 207.1036 - val__pinball_loss: 3.6458
Epoch 3/1000
1309/1309 [==============================] - 11s 8ms/step - loss: 2.7306 - mean_absolute_error: 5.4613 - mean_squared_error: 137.5288 - _pinball_loss: 2.7306 - val_loss: 3.5977 - val_mean_absolute_error: 7.1954 - val_mean_squared_error: 205.4731 - val__pinball_loss: 3.5979
Epoch 4/1000
1309/1309 [==============================] - 11s 8ms/step - loss: 2.6791 - mean_absolute_error: 5.3582 - mean_squared_error: 135.8095 - _pinball_loss: 2.6791 - val_loss: 3.5601 - val_mean_absolute_error: 7.1201 - val_mean_squared_error: 197.9437 - val__pinball_loss: 3.5597
Epoch 5/1000
1309/1309 [==============================] - 11s 8ms/step - loss: 2.6483 - mean_absolute_error: 5.2967 - mean_squared_error: 134.1014 - _pi

...

1309/1309 [==============================] - 11s 8ms/step - loss: 1.7671 - mean_absolute_error: 5.2931 - mean_squared_error: 169.9728 - _pinball_loss: 1.7671 - val_loss: 2.5737 - val_mean_absolute_error: 7.6558 - val_mean_squared_error: 270.4462 - val__pinball_loss: 2.5740
Epoch 33/1000
1309/1309 [==============================] - 11s 8ms/step - loss: 1.7653 - mean_absolute_error: 5.2886 - mean_squared_error: 170.0795 - _pinball_loss: 1.7653 - val_loss: 2.5878 - val_mean_absolute_error: 7.8186 - val_mean_squared_error: 279.0526 - val__pinball_loss: 2.5874
Epoch 34/1000
1309/1309 [==============================] - 11s 8ms/step - loss: 1.7666 - mean_absolute_error: 5.3011 - mean_squared_error: 170.7094 - _pinball_loss: 1.7666 - val_loss: 2.5887 - val_mean_absolute_error: 7.8466 - val_mean_squared_error: 280.5967 - val__pinball_loss: 2.5885
Epoch 35/1000
1309/1309 [==============================] - 11s 8ms/step - loss: 1.7575 - mean_absolute_error: 5.2764 - mean_squared_error: 170.0500 - 

1309/1309 [==============================] - 11s 8ms/step - loss: 1.4135 - mean_absolute_error: 6.0712 - mean_squared_error: 196.1459 - _pinball_loss: 1.4135 - val_loss: 1.9884 - val_mean_absolute_error: 8.3931 - val_mean_squared_error: 306.8586 - val__pinball_loss: 1.9878
Epoch 15/1000
1309/1309 [==============================] - 11s 8ms/step - loss: 1.4055 - mean_absolute_error: 6.0526 - mean_squared_error: 196.0869 - _pinball_loss: 1.4055 - val_loss: 1.9876 - val_mean_absolute_error: 8.5923 - val_mean_squared_error: 320.8597 - val__pinball_loss: 1.9876
Epoch 16/1000
1309/1309 [==============================] - 11s 8ms/step - loss: 1.3965 - mean_absolute_error: 6.0264 - mean_squared_error: 195.5986 - _pinball_loss: 1.3965 - val_loss: 1.9811 - val_mean_absolute_error: 8.6238 - val_mean_squared_error: 320.6826 - val__pinball_loss: 1.9810
Epoch 17/1000
1309/1309 [==============================] - 11s 8ms/step - loss: 1.3890 - mean_absolute_error: 6.0169 - mean_squared_error: 195.8154 - 

1309/1309 [==============================] - 11s 9ms/step - loss: 1.2905 - mean_absolute_error: 5.7365 - mean_squared_error: 192.0522 - _pinball_loss: 1.2905 - val_loss: 1.8936 - val_mean_absolute_error: 8.1222 - val_mean_squared_error: 302.1047 - val__pinball_loss: 1.8938
Epoch 43/1000
1309/1309 [==============================] - 11s 9ms/step - loss: 1.2849 - mean_absolute_error: 5.7246 - mean_squared_error: 192.0155 - _pinball_loss: 1.2849 - val_loss: 1.8841 - val_mean_absolute_error: 8.3369 - val_mean_squared_error: 314.7753 - val__pinball_loss: 1.8846
Epoch 44/1000
1309/1309 [==============================] - 11s 9ms/step - loss: 1.2819 - mean_absolute_error: 5.7184 - mean_squared_error: 191.9883 - _pinball_loss: 1.2819 - val_loss: 1.8889 - val_mean_absolute_error: 8.1343 - val_mean_squared_error: 302.5463 - val__pinball_loss: 1.8894
Epoch 45/1000
1309/1309 [==============================] - 11s 9ms/step - loss: 1.2802 - mean_absolute_error: 5.7119 - mean_squared_error: 191.8347 - 

1309/1309 [==============================] - 11s 9ms/step - loss: 0.8960 - mean_absolute_error: 7.5170 - mean_squared_error: 238.0247 - _pinball_loss: 0.8960 - val_loss: 1.2341 - val_mean_absolute_error: 10.2421 - val_mean_squared_error: 382.4653 - val__pinball_loss: 1.2342
Epoch 11/1000
1309/1309 [==============================] - 11s 9ms/step - loss: 0.8832 - mean_absolute_error: 7.4349 - mean_squared_error: 235.4435 - _pinball_loss: 0.8832 - val_loss: 1.2258 - val_mean_absolute_error: 9.6068 - val_mean_squared_error: 358.0888 - val__pinball_loss: 1.2257
Epoch 12/1000
1309/1309 [==============================] - 11s 9ms/step - loss: 0.8636 - mean_absolute_error: 7.2962 - mean_squared_error: 232.8747 - _pinball_loss: 0.8636 - val_loss: 1.2172 - val_mean_absolute_error: 10.3084 - val_mean_squared_error: 381.0116 - val__pinball_loss: 1.2170
Epoch 13/1000
1309/1309 [==============================] - 11s 9ms/step - loss: 0.8572 - mean_absolute_error: 7.2931 - mean_squared_error: 234.0537 

1309/1309 [==============================] - 11s 9ms/step - loss: 0.7288 - mean_absolute_error: 6.4878 - mean_squared_error: 217.9215 - _pinball_loss: 0.7288 - val_loss: 1.1060 - val_mean_absolute_error: 8.6898 - val_mean_squared_error: 329.7236 - val__pinball_loss: 1.1059
Epoch 39/1000
1309/1309 [==============================] - 11s 9ms/step - loss: 0.7273 - mean_absolute_error: 6.4775 - mean_squared_error: 217.3483 - _pinball_loss: 0.7273 - val_loss: 1.0887 - val_mean_absolute_error: 8.7986 - val_mean_squared_error: 331.6814 - val__pinball_loss: 1.0884
Epoch 40/1000
1309/1309 [==============================] - 11s 9ms/step - loss: 0.7247 - mean_absolute_error: 6.4555 - mean_squared_error: 216.7871 - _pinball_loss: 0.7247 - val_loss: 1.0992 - val_mean_absolute_error: 8.9316 - val_mean_squared_error: 339.8755 - val__pinball_loss: 1.0991
Epoch 41/1000
1309/1309 [==============================] - 11s 9ms/step - loss: 0.7246 - mean_absolute_error: 6.4618 - mean_squared_error: 216.7753 - 

In [3]:
import json
experiment_setting["total_error"] = history["total_error"] / (36 * df.shape[0] * 0.2)
experiment_setting["total_submission_error"] = history["total_submission_error"] / (18 * 80 * 96)

with open(f"experiments/cnn/{setting_index}.json", "w") as f:
    json.dump(experiment_setting, f, indent=4)

In [4]:
print(history["total_error"]/ (36 * df.shape[0] * 0.2), history["total_submission_error"] / (18 * 80 * 96))

0.00010064181552136104 2.3593061802991033


In [5]:
submission_df = load_submission_data()

result_list = history["submission_df"]
for result_df in result_list:
    for i in range(1, 10):
        q = i/10
        submission_df[f"q_{q}"] += result_df[f"q_{q}"]
                      
for i in range(1, 10):
    q = i/10
    submission_df[f"q_{q}"] = submission_df[f"q_{q}"] / (index + 1)

In [6]:
submission_df.to_csv(f"cnn_split_dataset_cnn_setting_index_{setting_index}.csv", index=False)

In [7]:
142800 / (48 * 81)

36.72839506172839

In [8]:
680400 / ( 48 * 81 * 5)

35.0